In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding, InputLayer
import string

In [2]:
pip install keras

In [3]:
# df = pd.read_csv("train.csv")
# len(df)

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/CostiCTI/CourseML/refs/heads/main/Part2-Models/train.csv")

In [3]:
data = list(df['text'])

In [4]:
def generate_subsentences(sentences: list[str]) -> list[str]:
    result = []
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    for sentence in sentences:
        try:
            clean_sentence = sentence.translate(translator)
            words = clean_sentence.split()
            for length in range(2, 5):
                for start in range(len(words) - length + 1):
                    subsentence = ' '.join(words[start:start + length])
                    result.append(subsentence)
            result.append("<start>" + " " + sentence.split(" ")[0])
        except:
            pass
    return result

#sentences = ["the cat sat on the table", "i like it"]
#print(generate_subsentences(sentences))

In [5]:
props = generate_subsentences(data)
print (len(props))

4090603


In [6]:
import random

random.shuffle(props)

In [7]:
props[0]

'chiar il merita scenariul'

In [8]:
props = props[:500000]
print (len(props))

500000


In [9]:
tokenizer = Tokenizer(num_words=3200, oov_token='unktoken')
tokenizer.fit_on_texts(props)

In [10]:
len(tokenizer.index_word)

43703

In [11]:
tokenizer.index_word[1]

'unktoken'

In [12]:
NO_WORDS = 3200

In [13]:
oftenit = []
for k, v in tokenizer.index_word.items():
    if k <= NO_WORDS:
        oftenit.append(v)
print (len(oftenit))

3200


In [14]:
len(oftenit)

3200

In [15]:
#tokenizer.index_word

In [16]:
props[10]

'putin eu vreau ceva'

In [17]:
sequences = tokenizer.texts_to_sequences(props)

In [18]:
sequences[10]

[96, 143, 272, 103]

In [19]:
sequences[124]

[48, 67, 4]

In [20]:
xsequences = []
for seq in sequences:
    if len(seq) > 5 or seq[-1] == 1:
        pass
    else:
        xsequences.append(seq)
print (len(xsequences))

429731


In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

padded = pad_sequences(xsequences, padding='pre')

In [22]:
padded[124]

array([2692,   19,    9,    6], dtype=int32)

In [23]:
print (padded[32])
print (padded[100])
print (padded[124])

[1462    8  196    2]
[  0   0   3 208]
[2692   19    9    6]


In [24]:
len(padded)

429731

In [25]:
X, y = padded[:,:-1], padded[:,-1]
y = to_categorical(y, num_classes=NO_WORDS + 1)

In [27]:
from sklearn.model_selection import train_test_split
X_train = X[:350000]
X_test = X[350000:]
y_train = y[:350000]
y_test = y[350000:]
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(350000, 3)
(350000, 3201)
(79731, 3)
(79731, 3201)


In [28]:
def top_3_accuracy(y_true, y_pred):
    y_true = tf.cast(tf.argmax(y_true, axis=-1), tf.int32)  # Convert one-hot to integer labels
    top_3 = tf.math.top_k(y_pred, k=3).indices
    matches = tf.reduce_any(tf.equal(tf.expand_dims(y_true, -1), top_3), axis=-1)
    return tf.reduce_mean(tf.cast(matches, tf.float32))

In [30]:
model = Sequential()
model.add(InputLayer(input_shape=(3, ), dtype=np.int32))
model.add(Embedding(NO_WORDS + 1, 16, input_length=4))
model.add(LSTM(16))
model.add(Dense(NO_WORDS + 1, activation='softmax'))
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 3, 16)               │          51,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           2,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3201)                │          54,417 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 107,745 (420.88 KB)

 Trainable params: 107,745 (420.88 KB)

 Non-trainable params: 0 (0.00 B)

None


In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', top_3_accuracy])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=4, validation_data=(X_test, y_test))

In [ ]:
preds = model.predict(X_test)

In [ ]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
results = []
wordsr = []
for pred in preds:
    ar = pred.argsort()[-5:][::-1]
    results.append([ar[0], ar[1], ar[2], ar[3], ar[4]])
    wordsr.append([reverse_word_map[ar[0]], reverse_word_map[ar[1]], reverse_word_map[ar[2]], reverse_word_map[ar[3]], reverse_word_map[ar[4]]])

In [ ]:
testy = [np.argmax(x) for x in y_test]

In [ ]:
len(testy)

In [ ]:
acc1 = 0
acc2 = 0
acc3 = 0
for i in range(len(results)):
    if results[i][0] == testy[i]:
        acc1 += 1
    if testy[i] in results[i][:2]:
        acc2 += 1
    if testy[i] in results[i][:3]:
        acc3 += 1
print ('R1:', acc1 / len(testy))
print ('R2:', acc2 / len(testy))
print ('R3:', acc3 / len(testy))

In [ ]:
aux = "salut ce faci in"
example = "imi place"

In [ ]:
example_seq = tokenizer.texts_to_sequences([aux, example])
print (example_seq)

In [ ]:
example_padded = pad_sequences(example_seq, padding='pre')
print (example_padded)

In [ ]:
pred = model.predict(example_padded)[1]

In [ ]:
ar = pred.argsort()[-3:][::-1]
res = [ar[0], ar[1], ar[2]]
words_pred = [reverse_word_map[ar[0]], reverse_word_map[ar[1]], reverse_word_map[ar[2]]]

In [ ]:
res

In [ ]:
words_pred